In [4]:
import glob
import librosa
import numpy as np
import random

In [5]:
root = './free-spoken-digit-dataset-master/'
files = sorted(glob.glob(root + '**/*.wav', recursive=True))
print(len(files))

3000


In [25]:
def preprocess_audio(path):
    data, sr = librosa.load(path, sr=48000)
    data, _ = librosa.effects.trim(data, top_db=10)
    data = librosa.util.fix_length(data, size=24000)
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)
    mfcc_T = mfcc.T
    return mfcc_T.astype(np.float32)

In [26]:
random_number = random.randint(1,3000)
print(files[random_number])
mfcc = preprocess_audio(files[random_number])

./free-spoken-digit-dataset-master/recordings/7_yweweler_7.wav


In [28]:
mfcc = mfcc.flatten()

In [29]:
def write_cpp_header(array, filename, var_name):
    flat = array.flatten()
    size = flat.size
    with open(filename, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f"static const int {var_name}_size = {size};\n\n")
        f.write(f"static const float {var_name}[{size}] = {{\n")
        for i, x in enumerate(flat):
            if i % 10 == 0:
                f.write("    ")
            f.write(f"{x:.6f}f")
            if i != size - 1:
                f.write(", ")
            if (i + 1) % 10 == 0:
                f.write("\n")
        if size % 10 != 0:
            f.write("\n")
        f.write("};\n")

write_cpp_header(mfcc, "mfcc_data.h", "mfcc_data")